In [50]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from matplotlib import pyplot as plt
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers import Embedding,LSTM,GRU,TimeDistributed,RepeatVector,Merge,BatchNormalization

from keras.optimizers import SGD, RMSprop, Adam

from sklearn.preprocessing import MinMaxScaler
import math

import cPickle as pickle

from keras.callbacks import RemoteMonitor
from sklearn.preprocessing import MinMaxScaler

In [2]:
base_path = "/home/docker/fastai-courses/deeplearning1/nbs/persistent/dmc/deep-learning-time-series-analysis/"
resources_folder = "resources/"

In [3]:
data = pd.read_csv('../data/train.csv', sep='|')
data['itemsPurchased'] = (data['revenue'] / data['price']).astype(int)
data['purchase'] = data['itemsPurchased'].apply(lambda x: 1 if x >= 1 else int(x))
data

,lineID,day,pid,adFlag,availability,competitorPrice,click,basket,order,price,revenue,itemsPurchased,purchase
0,1,1,6570,0,2,14.60,1,0,0,16.89,0.00,0,0
1,2,1,14922,1,1,8.57,0,1,0,8.75,0.00,0,0
2,3,1,16382,0,1,14.77,0,1,0,16.06,0.00,0,0
3,4,1,1145,1,1,6.59,0,0,1,6.55,6.55,1,1
4,5,1,3394,0,1,4.39,0,0,1,4.14,4.14,1,1
5,6,1,3661,0,1,13.66,0,0,1,10.03,10.03,1,1
6,7,1,3856,1,1,3.03,0,0,1,3.58,3.58,1,1
7,8,1,16963,0,1,8.78,1,0,0,8.75,0.00,0,0
8,9,1,14560,0,1,10.84,1,0,0,12.04,0.00,0,0
9,10,1,4853,1,1,9.12,1,0,0,8.75,0.00,0,0


In [12]:
# data[['lineID','itemsPurchased']].groupby(['itemsPurchased']).agg(['count'])

In [4]:
nr_days = data['day'].max() +1 

In [5]:
nr_train_days = nr_days
nr_test_days = nr_days 

# nr_train_days = 70
# nr_test_days = nr_days - 71

# train_data = data[data['day'] <= nr_train_days]
# test_data = data[data['day'] > nr_train_days]

In [6]:
# print("Train day interval: %d -> %d"%(train_data['day'].min(),train_data['day'].max()))
# print("Test day interval: %d -> %d"%(test_data['day'].min(),test_data['day'].max()))
print("Train days: %d"%nr_train_days)
print("Test days: %d"%nr_test_days)

Train days: 93
Test days: 93


In [7]:
def get_dict_for_pid(df, pid):
    filtered = df.loc[df['pid'] == pid].set_index('day')
    return filtered.to_dict(orient='index')

def save_obj(obj, path ):
    with open(path, 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(path):
    with open(path, 'rb') as f:
        return pickle.load(f)
    
def get_train_test_timeseries_for_data(data):

    uniquePids = sorted(data['pid'].unique())

    total = len(uniquePids)
    nr_train = int(total * 0.7)
    nr_test = int(total * 0.3)
    print("%d %d "%(nr_train,nr_test))
    
    train_pids = uniquePids[:nr_train]
    test_pids = uniquePids[nr_train:nr_train+nr_test]
    
    train_timeseries = [get_dict_for_pid(data, pid) for pid in tqdm(train_pids)]
    test_timeseries = [get_dict_for_pid(data, pid) for pid in tqdm(test_pids)]
    
    return (train_timeseries,test_timeseries)

In [13]:
(train_timeseries,test_timeseries) = get_train_test_timeseries_for_data(data)

  0%|          | 11/15349 [00:00<02:44, 93.34it/s]

15349 6578 


100%|██████████| 6578/6578 [01:12<00:00, 91.12it/s] 


In [14]:
save_obj(train_timeseries,base_path+resources_folder+"train_timeseries")
save_obj(test_timeseries,base_path+resources_folder+"test_timeseries")

In [31]:
train_timeseries  = load_obj(base_path+resources_folder+"train_timeseries")
test_timeseries = load_obj(base_path+resources_folder+"test_timeseries")

In [32]:
train_timeseries[0]

{10: {'adFlag': 0.0,
  'availability': 3.0,
  'basket': 0.0,
  'click': 1.0,
  'competitorPrice': 8.2200000000000006,
  'itemsPurchased': 0.0,
  'lineID': 198133.0,
  'order': 0.0,
  'pid': 1.0,
  'price': 10.08,
  'purchase': 0.0,
  'revenue': 0.0},
 21: {'adFlag': 0.0,
  'availability': 3.0,
  'basket': 0.0,
  'click': 1.0,
  'competitorPrice': 8.7100000000000009,
  'itemsPurchased': 0.0,
  'lineID': 430014.0,
  'order': 0.0,
  'pid': 1.0,
  'price': 10.08,
  'purchase': 0.0,
  'revenue': 0.0},
 33: {'adFlag': 0.0,
  'availability': 3.0,
  'basket': 0.0,
  'click': 0.0,
  'competitorPrice': 8.2200000000000006,
  'itemsPurchased': 1.0,
  'lineID': 769328.0,
  'order': 1.0,
  'pid': 1.0,
  'price': 7.2800000000000002,
  'purchase': 1.0,
  'revenue': 7.2800000000000002},
 40: {'adFlag': 0.0,
  'availability': 3.0,
  'basket': 0.0,
  'click': 1.0,
  'competitorPrice': 8.2200000000000006,
  'itemsPurchased': 0.0,
  'lineID': 1011505.0,
  'order': 0.0,
  'pid': 1.0,
  'price': 8.4199999999

In [108]:
nr_train_examples = len(train_timeseries)
nr_test_examples = len(test_timeseries)
nr_features = 5

print("Nr Train Examples %d"%nr_train_examples)
print("Nr Test Examples %d"%nr_test_examples)

Nr Train Examples 15349
Nr Test Examples 6578


In [129]:
count_by_day_df = pd.read_csv('../data/count-by-day.csv')[['itemsPurchased','day']]
count_by_day_df = count_by_day_df.set_index('day')
count_by_day_df

,itemsPurchased
day,
1,8736
2,8292
3,13235
4,13202
5,13066
6,13132
7,9934
8,8347
9,6869


In [130]:
def computeFeatureMatrix(timeseries,number_examples):
    
    data_matrix = np.zeros((number_examples,nr_days,nr_features))
    data_matrix.shape

    last_known_day = np.zeros(nr_features)
    for index in tqdm(range(number_examples)):
#     for index in range(number_examples):
        current_dict = timeseries[index]
        for day in range(nr_days):
            if day in current_dict:
                day_dict = current_dict[day]
                adFlag = day_dict['adFlag']
                availability = day_dict['availability']
                price = day_dict['price']

                if(math.isnan(day_dict['competitorPrice'])):
                    competitiorPrice = price
                else:
                    competitorPrice = day_dict['competitorPrice']
                
                day_sum = count_by_day_df.loc[day]
                
                last_known_day = [price,competitorPrice,availability,adFlag,day_sum]


            data_matrix[index,day] = last_known_day
            
    return data_matrix

In [131]:
train_matrix = computeFeatureMatrix(train_timeseries,nr_train_examples)
test_matrix = computeFeatureMatrix(test_timeseries,nr_test_examples)

print(train_matrix.shape)
print(test_matrix.shape)





  0%|          | 0/15349 [00:00<?, ?it/s]



  0%|          | 11/15349 [00:00<02:41, 95.15it/s]



  0%|          | 21/15349 [00:00<02:40, 95.21it/s]



  0%|          | 36/15349 [00:00<02:23, 106.91it/s]



  0%|          | 49/15349 [00:00<02:16, 111.75it/s]



  0%|          | 67/15349 [00:00<02:02, 125.24it/s]



  1%|          | 79/15349 [00:00<02:11, 116.10it/s]



  1%|          | 92/15349 [00:00<02:07, 119.81it/s]



  1%|          | 107/15349 [00:00<02:00, 126.16it/s]



  1%|          | 120/15349 [00:01<02:17, 110.80it/s]



  1%|          | 134/15349 [00:01<02:10, 117.01it/s]



  1%|          | 147/15349 [00:01<02:07, 119.70it/s]



  1%|          | 160/15349 [00:01<02:07, 119.43it/s]



  1%|          | 173/15349 [00:01<02:06, 120.18it/s]



  1%|          | 187/15349 [00:01<02:03, 122.75it/s]



  1%|▏         | 200/15349 [00:01<02:07, 119.02it/s]



  1%|▏         | 218/15349 [00:01<01:55, 130.89it/s]



  2%|▏         | 232/15349 [00:01<01:56, 130.14it/s]



  2%|▏  

(15349, 93, 5)
(6578, 93, 5)


In [132]:
def computePurchaseMatrix(timeseries,number_examples):
    purchase_matrix = np.zeros((number_examples,nr_days,1))
    
    for index in range(number_examples):
        current_dict = timeseries[index]
        for day in current_dict.keys():
            day_dict = current_dict[day]
            purchase = day_dict['purchase']
            purchase_matrix[index,day,0] = purchase
        
    return purchase_matrix

In [133]:
# def computePurchaseMatrix_assumption(timeseries,number_examples):
#     purchase_matrix = np.zeros((number_examples,nr_days,1))
    
#     last_known_purchase = 0
    
#     for index in range(number_examples):
#         current_dict = timeseries[index]
#         for day in range(nr_days):
#             if day in current_dict:
#                 day_dict = current_dict[day]
#                 day_dict = current_dict[day]
#                 purchase = day_dict['purchase']
#                 last_known_purchase = purchase
               
#             purchase_matrix[index,day,0] = last_known_purchase

#     return purchase_matrix

In [134]:
train_purchase_matrix = computePurchaseMatrix(train_timeseries,nr_train_examples)
test_purchase_matrix = computePurchaseMatrix(test_timeseries,nr_test_examples)

print(train_purchase_matrix.shape)
print(test_purchase_matrix.shape)

(15349, 93, 1)
(6578, 93, 1)


In [135]:
def computeGroudTruthMatrix(timeseries,number_examples,number_days):
    groud_truth_matrix = np.zeros((number_examples,number_days,1))
    groud_truth_matrix.fill(-1)

    for index in range(number_examples):
        current_dict = timeseries[index]
        for day in current_dict.keys():
            day_dict = current_dict[day]
            items_purchase = day_dict['itemsPurchased']
            groud_truth_matrix[index,day,0] = items_purchase   
            
    
    return groud_truth_matrix

In [136]:
train_groud_truth_matrix = computeGroudTruthMatrix(train_timeseries,nr_train_examples,nr_train_days)
test_groud_truth_matrix = computeGroudTruthMatrix(test_timeseries,nr_test_examples,nr_test_days)
print(train_groud_truth_matrix.shape)
print(test_groud_truth_matrix.shape)

(15349, 93, 1)
(6578, 93, 1)


In [137]:
train_shape = train_matrix.shape
test_shape = test_matrix.shape

print(train_shape)
print(test_shape)

(15349, 93, 5)
(6578, 93, 5)


In [138]:
# scaler = MinMaxScaler(feature_range=(0, 1))
# scaled_train_matrix = scaler.fit_transform(train_matrix.flatten())
# scale_test_matrix = scaler.fit_transform(test_matrix.flatten())

# scaled_train_matrix = scaled_train_matrix.reshape(train_shape)
# scale_test_matrix = scale_test_matrix.reshape(test_shape)

# print(scaled_train_matrix.shape)
# print(scale_test_matrix.shape)

In [139]:
final_train_matrix = train_matrix
final_test_matrix = test_matrix

# final_train_matrix = scaled_train_matrix
# final_test_matrix = scale_test_matrix


In [140]:
model = Sequential()
model.add(BatchNormalization(input_shape=(nr_days,nr_features)))
# model.add(LSTM(2048, return_sequences=True))
model.add(LSTM(1024, return_sequences=True))
model.add(TimeDistributed(Dense(1)))
model.compile(loss='mean_squared_error', optimizer = Adam(0.001))

In [155]:
model.fit(final_train_matrix, train_purchase_matrix, 
          validation_data = (final_test_matrix,test_purchase_matrix),    
          nb_epoch=10, 
          batch_size = 256,
          callbacks = [RemoteMonitor()],
          verbose = 1)

Train on 15349 samples, validate on 6578 samples
Epoch 1/10
15349/15349 [==============================] - 28s - loss: 0.0718 - val_loss: 0.0708

In [156]:
train_error = model.evaluate(final_train_matrix, train_purchase_matrix)
test_error = model.evaluate(final_test_matrix, test_purchase_matrix)


6578/6578 [==============================] - 16s     

In [157]:
print("Train MSE loss %f"%train_error)
print("Test MSE loss %f"%test_error)

Train MSE loss 0.068630
Test MSE loss 0.067785


In [158]:
models_folder = "models/"
save_path = base_path + models_folder
model.save(save_path+'bn_1-lstm-2048_20e_daySum.h5')

# Compute DMC Error 

In [150]:
train_predictions = model.predict(train_matrix)
test_predictions = model.predict(test_matrix)

print(train_predictions.shape)
print(test_predictions.shape)

(15349, 93, 1)
(6578, 93, 1)


In [151]:
train_predictions_arr = train_predictions.flatten()
test_predictions_arr = test_predictions.flatten()

train_groud_truth_arr = train_groud_truth_matrix.flatten().astype(int)
test_groud_truth_arr = test_groud_truth_matrix.flatten().astype(int)

train_days_mask = np.asarray([x != -1 for x in train_groud_truth_arr])
test_days_mask = np.asarray([x != -1 for x in test_groud_truth_arr])

print("%s %s %s"%(train_predictions_arr.shape,train_groud_truth_arr.shape,train_days_mask.shape ))
print("%s %s %s"%(test_predictions_arr.shape,test_groud_truth_arr.shape,test_days_mask.shape ))


(1427457,) (1427457,) (1427457,)
(611754,) (611754,) (611754,)


In [152]:
train_predictions_filtered = train_predictions_arr[train_days_mask]
train_groud_truth_filtered = train_groud_truth_arr[train_days_mask]

test_predictions_filtered = test_predictions_arr[test_days_mask]
test_groud_truth_filtered = test_groud_truth_arr[test_days_mask]

print("%s %s"%(train_predictions_filtered.shape,train_groud_truth_filtered.shape))
print("%s %s"%(test_predictions_filtered.shape,test_groud_truth_filtered.shape))


(520671,) (520671,)
(231167,) (231167,)


In [153]:
train_error = math.sqrt(((train_predictions_filtered - train_groud_truth_filtered)**2).sum())
test_error = math.sqrt(((test_predictions_filtered - test_groud_truth_filtered)**2).sum())

In [154]:
print("Train DMC error: %f"%train_error)
print("Test DMC error: %f"%test_error)


Train DMC error: 795.908853
Test DMC error: 543.307403
